# Sto learning 


In [ ]:
import ninja
import laplacian
import utils
import model
import graphs
from laplacian import normalized_laplacian
from utils import set_device
import os
import sys
import time

# -------------Torch--------------------------
import torch
from torch.utils.data import DataLoader

# -------------Graph--------------------------
import dgl
import matplotlib.pyplot as plt

# -------------Personal Files-----------------
import sys
sys.path.insert(0, "../lib/")


# Autoreload
%load_ext autoreload
%autoreload 2

# GPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

device = set_device(verbose=True)

# MNIST

In [ ]:
# Downloads the dataset if not found locally
from dataset import check_mnist_dataset_exists, datasampler, MNIST_rand, MNISTDataset

_ = check_mnist_dataset_exists()

nb_selected_train_data = 640
nb_selected_test_data = 128

train_data, train_labels, test_data, test_labels = datasampler(
    nb_selected_train_data, nb_selected_test_data
)

In [ ]:
trainset = MNIST_rand(train_data, train_labels)
testset = MNIST_rand(test_data, test_labels)



# Model definition for training

In [ ]:
from model import Classifier, SanityCheck

In [ ]:
# model_parameters
in_features = 1
first_conv_out = 32
second_conv_out = 50
hidden_layer_classifier = 512
k = 10
n_classes = trainset.num_classes

net = Classifier(
    in_features, first_conv_out, second_conv_out, hidden_layer_classifier, n_classes, k
)

print(net)
if torch.cuda.is_available():
    net.to(device)

# Training pipe

In [ ]:
import torch.optim as optim
import torch.nn as nn
#
from utils import collate
#

data_loader = DataLoader(trainset, batch_size=128,
                         shuffle=True, collate_fn=collate)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr= 0.005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.90)

epoch_losses = []

## Training time

In [ ]:
from laplacian import normalized_laplacian

net.train()
#optimizer = optim.Adam(net.parameters(), lr=0.001)
for epoch in range(1):
    epoch_loss = 0
    t0 = time.time()
    for iter, (bg, label, signal) in enumerate(data_loader):
        if iter == 0:
            L = normalized_laplacian(bg).to(device)
            #t = time.time()
            #print('\n THIS /!\ \n')
            #plt.imshow(signal.view(-1, 28), cmap='gray_r')

        if torch.cuda.is_available():
            bg.to(device)

        prediction = net(bg, signal, L)

        if iter == 0:
            t = time.time()
            print("One Prediction:\n", prediction[0], 'len:',
                  prediction.size(), '\n label:', label[0])  # DEBUG

        loss = loss_func(prediction, label.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.detach().item()

    epoch_loss /= iter + 1
    
    scheduler.step()
    
    print(
        "Epoch {}, loss {:.4f}, in {:.2f}(s) ".format(
            epoch, epoch_loss, time.time() - t0
        )
    )
    epoch_losses.append(epoch_loss)

In [ ]:
plt.title("cross entropy averaged over minibatches")
plt.plot(epoch_losses)
plt.show()

In [ ]:
torch.cuda.empty_cache()

## Test Metrics

In [ ]:
# Change

net.eval()
# Convert a list of tuples to two lists
test_X, test_Y, test_S = map(list, zip(*testset))
test_bg = dgl.batch(test_X)
test_Y = torch.tensor(test_Y).float().view(-1, 1)
L = normalized_laplacian(test_bg)
test_S = torch.stack(test_S).view(-1)
probs_Y = torch.softmax(net(test_bg,test_S, L), 1)
sampled_Y = torch.multinomial(probs_Y, 1)
argmax_Y = torch.max(probs_Y, 1)[1].view(-1, 1)

print(
    "Accuracy of sampled predictions on the test set: {:.4f}%".format(
        (test_Y == sampled_Y.cpu().float()).sum().item() / len(test_Y) * 100
    )
)
print(
    "Accuracy of argmax predictions on the test set: {:4f}%".format(
        (test_Y == argmax_Y.cpu().float()).sum().item() / len(test_Y) * 100
    )
)

In [ ]:
# BATCHED DGL EXAMPLE
import sklearn
from sklearn.metrics import classification_report

print(sklearn.metrics.classification_report(test_Y.cpu(), argmax_Y.cpu()))

## Saving

In [ ]:
from utils import save_model

save_model("Model_6000_20ep_noooo", net)